In [1]:
import ROOT
import array as array
from array import array 
import os
import glob
import ROOT
from ROOT import TH1D, TH2D, TTree,TFile, TUnfoldSys, TUnfoldDensity, TUnfold, TDirectoryFile
from random import gauss
import numpy as np
import root_numpy as rtnpy
from root_numpy import *
import h5py
#from awkward import JaggedArray, Table
import matplotlib.pyplot as plt
import seaborn as sns

Welcome to JupyROOT 6.24/00


/cvmfs/sft.cern.ch/lcg/views/LCG_100/x86_64-centos7-gcc8-opt/lib/ROOT/_facade.py:150: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  return _orig_ihook(name, *args, **kwds)



### Get list of nanoskim files to run over and sum gen event weights (ie, n_events) for a given samples

In [2]:
%jsroot on

In [3]:
def getListOfFiles(dirName, histos=False):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    #To ensure event counts don't come up against  a histogram file and rather just the nanoskim
    if histos==False: ftype="nanoskim"
    else: ftype="histogram"
    listOfFile[:] = [x for x in listOfFile if "%s"%ftype in x]
    #print (listOfFile)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):            
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

#print (getListOfFiles('/pnfs/psi.ch/cms/trivcat/store/user/kadatta/jetObservables/ZZ_TuneCP5_13TeV-pythia8/jetObservables_Skimmer_UL17_ZZ_v1/201125_043451/0000/'))

In [34]:
import argparse, os, shutil, sys
import ROOT
from array import array
from collections import OrderedDict
sys.path.insert(0,'../test/')
from variables import nSubVariables_WSel, nSubVariables_topSel
sys.path.insert(0,'../python/')
import CMS_lumi as CMS_lumi
import tdrstyle as tdrstyle
sys.path.insert(0,'../../Skimmer/test/')
from datasets import checkDict, dictSamples

####gReset()
ROOT.gROOT.SetBatch()
ROOT.gROOT.ForceStyle()
tdrstyle.setTDRStyle()
ROOT.gStyle.SetOptStat(0)

colorPallete = [ 0, 2, 4, 8, 12, 28, 30 ]


def loadHistograms( samples, var, sel, sysUnc=[], isMC=True, addGenInfo=True, respOnly=False, lumi=1., year='2017',process='MC' ):
    """docstring for loadHistograms"""

    SYSUNC = [ '_nom' ] + [ s+u for u in ['Up', 'Down'] for s in sysUnc ]

    allHistos = {}
    for isam in samples:
        tmpList = [ 'recoJet'+var+syst+sel for syst in SYSUNC ]
        if isMC and addGenInfo: tmpList = tmpList + [ 'genJet'+var+sel, 'missgenJet'+var+sel, 'accepgenJet'+var+sel, 'truerecoJet'+var+'_nom'+sel, 'fakerecoJet'+var+'_nom'+sel ] + [ 'respJet'+var+syst+sel for syst in SYSUNC ]
        if respOnly: tmpList = [ 'respJet'+var+syst+sel for syst in SYSUNC ]
        
        for ih in tmpList:
        
            print ('Processing '+isam+' '+ih)
            if isMC:
                allHistos[isam+'_'+ih] = samples[isam][0].Get( 'jetObservables/'+ih )
                tmpIsam = 'TT' if isam.startswith('data') else isam
                MCScale = samples[isam][1]['XS'] * lumi / samples[isam][1][year][('nGenWeights' if process.startswith('data') else 'nGenWeights') ]
                if not ih.startswith('respJet'): allHistos[isam+'_'+ih].Scale( MCScale )
            else:
                if sel.startswith('_dijet') and process.startswith('data'):
                    tmpdataHistos = {}
                    for it in checkDict( 'JetHT', dictSamples )[year]['triggerList']:
                        tmpdataHistos[ it ] = samples[isam][0].Get( 'jetObservables/'+ih.replace( sel, '_'+it+sel ) )
                        tmpdataHistos[ it ].Scale( checkDict( 'JetHT', dictSamples )[year]['triggerList'][it] )
                    allHistos[ isam+'_'+ih ] = tmpdataHistos[next(iter(tmpdataHistos))].Clone()
                    allHistos[ isam+'_'+ih ].Reset()
                    for i in tmpdataHistos: allHistos[isam+'_'+ih].Add( tmpdataHistos[i] )
                else:
                    allHistos[isam+'_'+ih] = samples[isam][0].Get( 'jetObservables/'+ih )

            if len(variables[var]['bins'])==1:
                genBin = variables[var]['bins'][0]
                recoBin = variables[var]['bins'][0]/2
            else:
                genBin = variables[var]['bins']
                recoBin = np.sort(np.append( variables[var]['bins'], np.array([ (variables[var]['bins'][i]+variables[var]['bins'][i+1])/2 for i in range(len(variables[var]['bins'])-1) ]  ) ))

            if not ih.startswith('respJet'):
                if len(variables[var]['bins'])==1:
                    if ih.startswith(('recoJet','fake', 'true')):
                        allHistos[isam+'_'+ih+'_genBin'] = allHistos[isam+'_'+ih].Clone()
                        allHistos[isam+'_'+ih+'_genBin'].Rebin( genBin )
                        allHistos[isam+'_'+ih].Rebin( recoBin )
                    else: allHistos[isam+'_'+ih].Rebin( genBin )
                else:
                    if ih.startswith(('recoJet','fake','true')):
                        allHistos[isam+'_'+ih+'_genBin'] = allHistos[isam+'_'+ih].Clone()
                        allHistos[isam+'_'+ih+'_genBin'] = allHistos[isam+'_'+ih+'_genBin'].Rebin( len(genBin)-1, allHistos[isam+'_'+ih].GetName()+"_Rebin_genBin", array( 'd', genBin ) )
                        allHistos[isam+'_'+ih] = allHistos[isam+'_'+ih].Rebin( len(recoBin)-1, allHistos[isam+'_'+ih].GetName()+"_Rebin", array( 'd', recoBin ) )
                    else:
                        allHistos[isam+'_'+ih] = allHistos[isam+'_'+ih].Rebin( len(genBin)-1, allHistos[isam+'_'+ih].GetName()+"_Rebin", array( 'd', genBin ) )
            else:
                if len(variables[var]['bins'])==1: allHistos[isam+'_'+ih].Rebin2D( genBin, recoBin )
                else:

                    #### fancy way to create variable binning TH2D
                    tmpHisto = ROOT.TH2F( allHistos[isam+'_'+ih].GetName()+isam+"_Rebin", allHistos[isam+'_'+ih].GetName()+isam+"_Rebin", len(genBin)-1, array( 'd', genBin), len(recoBin)-1, array( 'd', recoBin) )

                    tmpArrayContent = np.zeros((len(genBin), len(recoBin)))
                    tmpArrayError = np.zeros((len(genBin), len(recoBin)))

#                    for biny in range( 1, allHistos[isam+'_'+ih].GetNbinsY()+1 ):
#                        for binx in range( 1, allHistos[isam+'_'+ih].GetNbinsX()+1 ):
#                            tmpHisto.Fill( allHistos[isam+'_'+ih].GetXaxis().GetBinCenter(binx), allHistos[isam+'_'+ih].GetYaxis().GetBinCenter(biny), allHistos[isam+'_'+ih].GetBinContent( binx, biny ) )

                    for biny in range( 1, allHistos[isam+'_'+ih].GetNbinsY()+1 ):
                        by = allHistos[isam+'_'+ih].GetYaxis().GetBinCenter( biny )
                        for binx in range( 1, allHistos[isam+'_'+ih].GetNbinsX()+1 ):
                            bx = allHistos[isam+'_'+ih].GetXaxis().GetBinCenter(binx)
                            for iX in range( len(genBin)-1 ):
                                for iY in range( len(recoBin)-1 ):
                                    if (bx<genBin[iX+1] and bx>genBin[iX]) and (by<recoBin[iY+1] and by>recoBin[iY]):
                                        jbin = allHistos[isam+'_'+ih].GetBin(binx,biny)
                                        tmpArrayContent[iX][iY] = tmpArrayContent[iX][iY] + allHistos[isam+'_'+ih].GetBinContent( jbin )
                                        tmpArrayError[iX][iY] = tmpArrayError[iX][iY] + ROOT.TMath.Power( allHistos[isam+'_'+ih].GetBinError( jbin ), 2 )

                    for biny in range( 1, tmpHisto.GetNbinsY()+1 ):
                        for binx in range( 1, tmpHisto.GetNbinsX()+1 ):
                            #if (binx <= ((biny+1)/2.)+3) and (binx >= ((biny+1)/2.)-3):
                            tmpHisto.SetBinContent( tmpHisto.GetBin(binx,biny), tmpArrayContent[binx-1][biny-1] )
                            tmpHisto.SetBinError( tmpHisto.GetBin(binx,biny), ROOT.TMath.Sqrt(tmpArrayError[binx-1][biny-1] ) )

                    tmpHisto.Sumw2()
                    allHistos[isam+'_'+ih] = tmpHisto

                if isMC: allHistos[isam+'_'+ih].Scale( MCScale )
                ##### For tests, projections directly from 2D
                allHistos[isam+'_genJetfrom_'+ih] = allHistos[isam+'_'+ih].ProjectionY()
                allHistos[isam+'_recoJetfrom_'+ih] = allHistos[isam+'_'+ih].ProjectionX()


    tmpHistos = { k:v for (k,v) in allHistos.items() if 'Inf' in k }
    for ih in tmpHistos:
        for jh in allHistos:
            if (jh.endswith('0'+ih.split('Inf')[1])) and not ('Inf' in jh ):
                tmpHistos[ih].Add( allHistos[jh].Clone() )
    if len(tmpHistos)>0: allHistos = tmpHistos

    return allHistos


def runPurity( name, inputFiles, outputFolder, variables, sel, selection, year, version, ext, lumi, process='MC', only=False  ):
    """docstring for createDataCards"""

    numBins = 0
    numBinsList = [0]
    dictHistos = OrderedDict()
    for ivar in variables:
        print (ivar.split('_')[0])
        if not name.endswith( ivar.split('_')[0] ): continue
        ### Getting input histos
        signalHistos = loadHistograms( inputFiles, ivar, sel, sysUnc=[], lumi=lumi, year=year, process=process )
        outputDir=outputFolder
        if not os.path.exists(outputDir): os.makedirs(outputDir)

        legend=ROOT.TLegend(0.15,0.80,0.90,0.90)
        legend.SetFillStyle(0)
        legend.SetBorderSize(0)
        legend.SetLineColor(0)

        legend.SetTextSize(0.04)
        legend.SetNColumns( 4 )
        print (signalHistos)
        dictHistos[ 'acceptanceGraph_'+ivar ] = signalHistos[signalLabel+'_accepgenJet'+ivar+selection].Clone()
        dictHistos[ 'acceptanceGraph_'+ivar ].SetLineWidth(2)
        dictHistos[ 'acceptanceGraph_'+ivar ].SetLineColor(colorPallete[1])
        dictHistos[ 'acceptanceGraph_'+ivar ].Divide(  signalHistos[signalLabel+'_genJet'+ivar+selection] )
        legend.AddEntry( dictHistos[ 'acceptanceGraph_'+ivar ], 'Acceptance', 'l' )

        dictHistos[ 'fakeGraph_'+ivar ] = signalHistos[signalLabel+'_fakerecoJet'+ivar+'_nom'+selection+'_genBin'].Clone()
        dictHistos[ 'fakeGraph_'+ivar ].SetLineWidth(2)
        dictHistos[ 'fakeGraph_'+ivar ].SetLineColor(colorPallete[2])
        dictHistos[ 'fakeGraph_'+ivar ].Divide(  signalHistos[signalLabel+'_recoJet'+ivar+'_nom'+selection+'_genBin'] )
        legend.AddEntry( dictHistos[ 'fakeGraph_'+ivar ], 'Fakerate', 'l' )

        #### create diagonal
        tmpHisto = signalHistos[signalLabel+'_respJet'+ivar+'_nom'+sel].Clone()
        tmpHisto.Rebin2D( 1, 2 )
        dictHistos[ 'purityGraph_'+ivar ] = signalHistos[signalLabel+'_recoJet'+ivar+'_nom'+selection+'_genBin'].Clone()
        dictHistos[ 'purityGraph_'+ivar ].Reset()
        dictHistos[ 'stabilityGraph_'+ivar ] = dictHistos[ 'purityGraph_'+ivar ].Clone()
        for ibin in range( 1, tmpHisto.GetNbinsX()+1 ):
            diagReco = tmpHisto.GetBinContent( tmpHisto.GetBin(ibin-1, ibin) ) + tmpHisto.GetBinContent( tmpHisto.GetBin(ibin, ibin) ) + tmpHisto.GetBinContent( tmpHisto.GetBin(ibin+1, ibin) )
            dictHistos[ 'purityGraph_'+ivar ].SetBinContent( ibin, diagReco )
            diagGen = tmpHisto.GetBinContent( tmpHisto.GetBin(ibin, ibin-1) ) + tmpHisto.GetBinContent( tmpHisto.GetBin(ibin, ibin) ) + tmpHisto.GetBinContent( tmpHisto.GetBin(ibin, ibin+1) )
            dictHistos[ 'stabilityGraph_'+ivar ].SetBinContent( ibin, diagGen )

        dictHistos[ 'purityGraph_'+ivar ].SetLineWidth(2)
        dictHistos[ 'purityGraph_'+ivar ].SetLineColor(colorPallete[3])
        dictHistos[ 'purityGraph_'+ivar ].Divide(  signalHistos[signalLabel+'_recoJet'+ivar+'_nom'+selection+'_genBin'] )
        legend.AddEntry( dictHistos[ 'purityGraph_'+ivar ], 'Purity', 'l' )

        dictHistos[ 'stabilityGraph_'+ivar ].SetLineWidth(2)
        dictHistos[ 'stabilityGraph_'+ivar ].SetLineColor(colorPallete[4])
        dictHistos[ 'stabilityGraph_'+ivar ].Divide(  signalHistos[signalLabel+'_genJet'+ivar+selection] )
        legend.AddEntry( dictHistos[ 'stabilityGraph_'+ivar ], 'Stability', 'l' )

        ##### removing tau21 and tau32 from total plots
        if not ivar.endswith(('21', '32')):
            numBins = numBins + dictHistos[ 'purityGraph_'+ivar ].GetNbinsX()
            numBinsList.append(numBins)

        multigraph = ROOT.THStack()
        for i in dictHistos:
            if i.endswith(ivar): multigraph.Add( dictHistos[i]  )

        ###### Plot unfolding results
        ROOT.gStyle.SetPadRightMargin(0.05)
        canvas = ROOT.TCanvas('canvas', 'canvas', 750, 500)

        multigraph.Draw("hist nostack")
        multigraph.GetXaxis().SetTitle( variables[ivar]['label'] )
        multigraph.GetYaxis().SetTitle( 'Percentage' )
        multigraph.SetMaximum( 1.1 )
        multigraph.GetYaxis().SetTitleOffset(0.8)

        legend.Draw()
        CMS_lumi.extraText = "Simulation Preliminary"
        CMS_lumi.lumi_13TeV = "13 TeV, "+ ( '2017+2018' if year.startswith('all') else year )
        CMS_lumi.relPosX = 0.11
        CMS_lumi.CMS_lumi(canvas, 4, 0)

        textBox=ROOT.TLatex()
        textBox.SetNDC()
        textBox.SetTextSize(0.04)
        textBox.SetTextFont(62) ### 62 is bold, 42 is normal
        #textBox.DrawLatex(0.65, 0.75, sel.split('Sel')[0].split('_')[1]+' Selection' )

        canvas.SaveAs(outputDir+ivar+'_'+signalLabel+sel+'_Purity_'+version+'.'+ext)
        if ext.startswith('pdf'):
            canvas.SaveAs(outputDir+ivar+'_'+signalLabel+sel+'_Purity_'+version+'.png')


        ########## Plot response matrix
        print ('|------> Cross check: plotting response matrix for signal')
        ROOT.gStyle.SetPadRightMargin(0.15)
        #ROOT.gStyle.SetPalette(ROOT.kGistEarth)
        #ROOT.TColor.InvertPalette()
        can2D = ROOT.TCanvas(ivar+'can2D', ivar+'can2D', 750, 500 )
        signalHistos[signalLabel+'_respJet'+ivar+'_nom'+sel].GetXaxis().SetTitle('Accepted Gen '+variables[ivar]['label'])
        signalHistos[signalLabel+'_respJet'+ivar+'_nom'+sel].GetYaxis().SetTitle('True Reco '+variables[ivar]['label'])
        signalHistos[signalLabel+'_respJet'+ivar+'_nom'+sel].GetYaxis().SetTitleOffset( 0.8 )
        signalHistos[signalLabel+'_respJet'+ivar+'_nom'+sel].Draw("colz")
        CMS_lumi.relPosX = 0.12
        CMS_lumi.CMS_lumi(can2D, 4, 0)
        can2D.SaveAs(outputDir+ivar+'_'+signalLabel+sel+'_responseMatrix'+version+'.'+ext)
        if ext.startswith('pdf'):
            can2D.SaveAs(outputDir+ivar+'_'+signalLabel+sel+'_responseMatrix'+version+'.png')

    if not only:

        outputFileName = name+'_purity_'+signalLabel+'_combinePlots_'+version+'.'+ext
        print('Processing.......', outputFileName)

        legend=ROOT.TLegend(0.20,0.80,0.60,0.90)
        legend.SetNColumns(2)
        legend.SetFillStyle(0)
        legend.SetTextSize(0.06)
        legend.SetBorderSize(0)
        
        dictHistos['purity'] = ROOT.TH1F('purity', 'purity', numBinsList[-1], 0, numBinsList[-1])
        legend.AddEntry( dictHistos[ 'purity' ], 'Purity', 'lep' )
        dictHistos['stability'] = ROOT.TH1F('stability', 'stability', numBinsList[-1], 0, numBinsList[-1])
        legend.AddEntry( dictHistos[ 'stability' ], 'Stability', 'lep' )

        tmpNbin = 0
        Xlabels = []
        for ivar in variables:
            if not name.endswith( ivar.split('_')[0] ): continue
            else:
                Xlabels.append( variables[ivar]['label'][1] )
                for ibin in range(1, dictHistos['purityGraph_'+ivar].GetNbinsX()+1):
                    tmpNbin = tmpNbin+1
                    dictHistos['purity'].SetBinContent( tmpNbin, dictHistos['purityGraph_'+ivar].GetBinContent(ibin) )
                    dictHistos['stability'].SetBinContent( tmpNbin, dictHistos['stabilityGraph_'+ivar].GetBinContent(ibin) )

        ROOT.gStyle.SetPadRightMargin(0.05)
        #ROOT.gStyle.SetPadLeftMargin(0.08)
        canvasoutputFileName = ROOT.TCanvas('c1purity', 'c1purity', 1400, 500 )
        ROOT.gStyle.SetPadTickX(0)
        #pad1.SetLogy()
        dictHistos['purity'].GetXaxis().SetNdivisions(100)
        dictHistos['purity'].GetYaxis().SetTitle( '#frac{1}{d#sigma} #frac{d#sigma}{d#tau_{X}}' )
        dictHistos['purity'].GetYaxis().SetTitleSize( 0.06 )
        dictHistos['purity'].GetYaxis().SetTitleOffset( 0.8 )
        #dictHistos['purity'].SetMaximum( dictHistos['stability'].GetMaximum()*1.2 )
        dictHistos['purity'].SetMaximum( 0.84 )
        dictHistos['purity'].SetMinimum( 0.36 )
        dictHistos['purity'].SetLineColor( ROOT.kBlack )
        dictHistos['purity'].SetLineWidth( 2 )

        dictHistos['stability'].SetLineColor( ROOT.kMagenta )
        dictHistos['stability'].SetLineWidth( 2 )
        dictHistos['purity'].Draw('hist')
        dictHistos[ 'stability' ].Draw('hist same')

        ### division lines
        lines = {}
        for i in numBinsList[1:-1]:
            lines[i] = ROOT.TGraph(2, array('d', [i,i]), array('d', [0, 200]) )
            lines[i].SetLineColor(ROOT.kGray)
            lines[i].Draw('same')

        CMS_lumi.lumiTextSize = 0.6
        CMS_lumi.relPosX = 0.07
        CMS_lumi.CMS_lumi( canvasoutputFileName, 4, 0)
        legend.Draw()

        textBox=ROOT.TLatex()
        #textBox.SetNDC()
        textBox.SetTextSize(0.06)
        textBox.SetTextAlign(12)
        textBoxList = {}
        for i in range(1, len(numBinsList)):
            textBoxList[i] = textBox.Clone()
            textBoxList[i].DrawLatex(numBinsList[i-1]+(numBinsList[i]-numBinsList[i-1])/2., 0.33, Xlabels[i-1] )

        outputDir=outputFolder
        canvasoutputFileName.SaveAs( outputDir+'/'+outputFileName )
        if ext.startswith('pdf'):
            canvasoutputFileName.SaveAs( outputDir+'/'+outputFileName.replace('pdf', 'png') )
        ROOT.gStyle.SetPadTickX(1)

In [35]:
sel= '_WSel'
process = 'data'
version = '_UL17'

In [36]:
signalLabelBegin = 'TTToSemiLeptonic'
signalLabel = 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8'
inputFolder = '../test/Samples/'

sigFiles = {}

for iy in ( ['all']):
    for isam in dictSamples:
        if not checkDict( isam, dictSamples )[iy]['skimmerHisto'].endswith('root'): continue
        if isam.startswith(signalLabelBegin) and ('erdON' not in isam) and ('hdamp' not in isam):
            sigFiles[isam] = [
                            ROOT.TFile.Open(inputFolder+checkDict( isam, dictSamples )[iy]['skimmerHisto'] ),
                            checkDict( isam, dictSamples )
                        ]
print (sigFiles)
variables=nSubVariables_WSel
runPurity(name='recoJet', outputFolder='../test/Results/WSel/UL17/BinningTests/', inputFiles=sigFiles, variables=nSubVariables_WSel, sel='_WSel', selection='_WSel', year=iy, version='v1', ext='pdf', lumi=1., process='MC')

KeyError: 'all'

In [37]:
signalLabelBegin = 'TTToSemiLeptonic'
signalLabel = 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8'
inputFolder = '../test/Samples/'

sigFiles = {}

for iy in ( ['2017']):
    for isam in dictSamples:
        if not checkDict( isam, dictSamples )[iy]['skimmerHisto'].endswith('root'): continue
        if isam.startswith(signalLabelBegin) and ('erdON' not in isam) and ('hdamp' not in isam):
            sigFiles[isam] = [
                            ROOT.TFile.Open(inputFolder+checkDict( isam, dictSamples )[iy]['skimmerHisto'] ),
                            checkDict( isam, dictSamples )
                        ]
print (sigFiles)
variables=nSubVariables_WSel
runPurity(name='recoJet', outputFolder='../test/Results/WSel/UL17/BinningTests/', inputFiles=sigFiles, variables=nSubVariables_WSel, sel='_WSel', selection='_WSel', year=iy, version='v1', ext='pdf', lumi=1., process='MC')

{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8': [<cppyy.gbl.TFile object at 0xb8f5190>, {'2017': {'miniAOD': ['/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL17MiniAODv2-106X_mc2017_realistic_v9-v1/MINIAODSIM'], 'nanoAOD': ['/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/kadatta-RunIISummer19UL17PFNanoAOD-106X_mc2017_realistic_v9-v1-7a1edb72314467730e458def0bc98536/USER'], 'skimmer': ['/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/kadatta-RunIISummer19UL17PFNanoAOD-106X_mc2017_realistic_v9-v1-7a1edb72314467730e458def0bc98536/USER'], 'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_2017UL.root', 'nevents': 1.0, 'nGenWeights': 32392787477.907993}, '2018': {'miniAOD': ['/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1/MINIAODSIM'], 'nanoAOD': ['/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/kadatta-RunIISummer19UL18PFNanoAOD-106X_upgrade2018_realistic_v16_L1v1-v1-754f8ab81f6f0298c5fa7c45094d30e4/USER'], 'ski


Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_1_2_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_1_2_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_1_2_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_1_2_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_1_2_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau_1_2_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau_1_2_nom_WSel
{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau_1_2_nom_WSel': <cppyy.gbl.TH1F object at 0xbc26c10>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau_1_2_nom_WSel_genBin': <cppyy.gbl.TH1F object at 0xbc239d0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_genJet_tau_1_2_WSel': <cppyy.gbl.TH1F object at 0xbc2c780>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_missgenJet_tau_1_2_WSel': <cppyy.gbl.

{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau_2_3_nom_WSel': <cppyy.gbl.TH1F object at 0xbc64ee0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau_2_3_nom_WSel_genBin': <cppyy.gbl.TH1F object at 0xbc73060>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_genJet_tau_2_3_WSel': <cppyy.gbl.TH1F object at 0xbc7a9a0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_missgenJet_tau_2_3_WSel': <cppyy.gbl.TH1F object at 0xbc7d8e0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_accepgenJet_tau_2_3_WSel': <cppyy.gbl.TH1F object at 0xbc78c40>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_truerecoJet_tau_2_3_nom_WSel': <cppyy.gbl.TH1F object at 0xbc79490>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_truerecoJet_tau_2_3_nom_WSel_genBin': <cppyy.gbl.TH1F object at 0xbc83630>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_fakerecoJet_tau_2_3_nom_WSel': <cppyy.gbl.TH1F object at 0xbc83010>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_fakerecoJet_tau_2_3_nom_WSel_genBin'


Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau32_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau32_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau32_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau32_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau32_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau32_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau32_nom_WSel
{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau32_nom_WSel': <cppyy.gbl.TH1F object at 0xeb92260>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau32_nom_WSel_genBin': <cppyy.gbl.TH1F object at 0xeb93390>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_genJet_tau32_WSel': <cppyy.gbl.TH1F object at 0xeb86c30>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_missgenJet_tau32_WSel': <cppyy.gbl.TH1F object at 0xeb880

Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file ../test/Results/WSel/UL17/BinningTests/_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_WSel_Purity_v1.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results/WSel/UL17/BinningTests/_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_WSel_Purity_v1.png has been created
Info in <TCanvas::Print>: pdf file ../test/Results/WSel/UL17/BinningTests/_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_WSel_responseMatrixv1.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results/WSel/UL17/BinningTests/_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_WSel_responseMatrixv1.png has been created
Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanv

In [38]:
signalLabelBegin = 'TTToSemiLeptonic'
signalLabel = 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8'
inputFolder = '../test/Samples/'

sigFiles = {}

for iy in ( ['2018']):
    for isam in dictSamples:
        if not checkDict( isam, dictSamples )[iy]['skimmerHisto'].endswith('root'): continue
        if isam.startswith(signalLabelBegin) and ('erdON' not in isam) and ('hdamp' not in isam):
            sigFiles[isam] = [
                            ROOT.TFile.Open(inputFolder+checkDict( isam, dictSamples )[iy]['skimmerHisto'] ),
                            checkDict( isam, dictSamples )
                        ]
print (sigFiles)
variables=nSubVariables_WSel
runPurity(name='recoJet', outputFolder='../test/Results/WSel/UL18/BinningTests/', inputFiles=sigFiles, variables=nSubVariables_WSel, sel='_WSel', selection='_WSel', year=iy, version='v1', ext='pdf', lumi=1., process='MC')

{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8': [<cppyy.gbl.TFile object at 0xb8f5190>, {'2017': {'miniAOD': ['/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL17MiniAODv2-106X_mc2017_realistic_v9-v1/MINIAODSIM'], 'nanoAOD': ['/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/kadatta-RunIISummer19UL17PFNanoAOD-106X_mc2017_realistic_v9-v1-7a1edb72314467730e458def0bc98536/USER'], 'skimmer': ['/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/kadatta-RunIISummer19UL17PFNanoAOD-106X_mc2017_realistic_v9-v1-7a1edb72314467730e458def0bc98536/USER'], 'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_2017UL.root', 'nevents': 1.0, 'nGenWeights': 32392787477.907993}, '2018': {'miniAOD': ['/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1/MINIAODSIM'], 'nanoAOD': ['/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/kadatta-RunIISummer19UL18PFNanoAOD-106X_upgrade2018_realistic_v16_L1v1-v1-754f8ab81f6f0298c5fa7c45094d30e4/USER'], 'ski

|------> Cross check: plotting response matrix for signal

Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau_1_2_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau_1_2_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau_1_2_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau_1_2_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau_1_2_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau_1_2_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau_1_2_nom_WSel
{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau_1_2_nom_WSel': <cppyy.gbl.TH1F object at 0xbd21530>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau_1_2_nom_WSel_genBin': <cppyy.gbl.TH1F object at 0xbd24b30>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_genJet_tau_1_2_WSel': <cppyy.gbl.TH1F object at 0xbd1e7b0>, 'TTToSemiLeptonic_TuneCP5_

{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau_2_3_nom_WSel': <cppyy.gbl.TH1F object at 0xbc7e1a0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau_2_3_nom_WSel_genBin': <cppyy.gbl.TH1F object at 0xbc7e590>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_genJet_tau_2_3_WSel': <cppyy.gbl.TH1F object at 0xbc77790>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_missgenJet_tau_2_3_WSel': <cppyy.gbl.TH1F object at 0xbc7c590>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_accepgenJet_tau_2_3_WSel': <cppyy.gbl.TH1F object at 0xbc73bd0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_truerecoJet_tau_2_3_nom_WSel': <cppyy.gbl.TH1F object at 0xbc74680>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_truerecoJet_tau_2_3_nom_WSel_genBin': <cppyy.gbl.TH1F object at 0xc872010>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_fakerecoJet_tau_2_3_nom_WSel': <cppyy.gbl.TH1F object at 0xc8715d0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_fakerecoJet_tau_2_3_nom_WSel_genBin'


Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 recoJet_tau32_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 genJet_tau32_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 missgenJet_tau32_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 accepgenJet_tau32_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 truerecoJet_tau32_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 fakerecoJet_tau32_nom_WSel
Processing TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8 respJet_tau32_nom_WSel
{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau32_nom_WSel': <cppyy.gbl.TH1F object at 0xd7978a0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau32_nom_WSel_genBin': <cppyy.gbl.TH1F object at 0xd79bb40>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_genJet_tau32_WSel': <cppyy.gbl.TH1F object at 0xd7a0500>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_missgenJet_tau32_WSel': <cppyy.gbl.TH1F object at 0xd79cd

Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file ../test/Results/WSel/UL18/BinningTests/_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_WSel_Purity_v1.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results/WSel/UL18/BinningTests/_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_WSel_Purity_v1.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: _tau_0p5_1can2D
Info in <TCanvas::Print>: pdf file ../test/Results/WSel/UL18/BinningTests/_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_WSel_responseMatrixv1.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results/WSel/UL18/BinningTests/_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_WSel_responseMatrixv1.png has been created
Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warni

In [39]:
signalLabelBegin = 'TTToSemiLeptonic'
signalLabel = 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8'
inputFolder = '../test/Samples/'

sigFiles = {}

for iy in ( ['2017']):
    for isam in dictSamples:
        if not checkDict( isam, dictSamples )[iy]['skimmerHisto'].endswith('root'): continue
        if isam.startswith(signalLabelBegin) and ('erdON' not in isam) and ('hdamp' not in isam):
            sigFiles[isam] = [
                            ROOT.TFile.Open(inputFolder+checkDict( isam, dictSamples )[iy]['skimmerHisto'] ),
                            checkDict( isam, dictSamples )
                        ]
print (sigFiles)
variables=nSubVariables_topSel
runPurity(name='recoJet', outputFolder='../test/Results/topSel/UL17/BinningTests/', inputFiles=sigFiles, variables=nSubVariables_WSel, sel='_topSel', selection='_topSel', year=iy, version='v1', ext='pdf', lumi=1., process='MC')

{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8': [<cppyy.gbl.TFile object at 0xbc05750>, {'2017': {'miniAOD': ['/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL17MiniAODv2-106X_mc2017_realistic_v9-v1/MINIAODSIM'], 'nanoAOD': ['/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/kadatta-RunIISummer19UL17PFNanoAOD-106X_mc2017_realistic_v9-v1-7a1edb72314467730e458def0bc98536/USER'], 'skimmer': ['/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/kadatta-RunIISummer19UL17PFNanoAOD-106X_mc2017_realistic_v9-v1-7a1edb72314467730e458def0bc98536/USER'], 'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_2017UL.root', 'nevents': 1.0, 'nGenWeights': 32392787477.907993}, '2018': {'miniAOD': ['/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1/MINIAODSIM'], 'nanoAOD': ['/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/kadatta-RunIISummer19UL18PFNanoAOD-106X_upgrade2018_realistic_v16_L1v1-v1-754f8ab81f6f0298c5fa7c45094d30e4/USER'], 'ski

{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau_0p5_2_nom_topSel': <cppyy.gbl.TH1F object at 0xf2e7e40>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau_0p5_2_nom_topSel_genBin': <cppyy.gbl.TH1F object at 0xc877bf0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_genJet_tau_0p5_2_topSel': <cppyy.gbl.TH1F object at 0xc878390>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_missgenJet_tau_0p5_2_topSel': <cppyy.gbl.TH1F object at 0xc8787f0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_accepgenJet_tau_0p5_2_topSel': <cppyy.gbl.TH1F object at 0xbd2b640>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_truerecoJet_tau_0p5_2_nom_topSel': <cppyy.gbl.TH1F object at 0xbc6abc0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_truerecoJet_tau_0p5_2_nom_topSel_genBin': <cppyy.gbl.TH1F object at 0xbc6afb0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_fakerecoJet_tau_0p5_2_nom_topSel': <cppyy.gbl.TH1F object at 0xbd2bf00>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_fake

{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau_1_3_nom_topSel': <cppyy.gbl.TH1F object at 0xbc0ed50>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau_1_3_nom_topSel_genBin': <cppyy.gbl.TH1F object at 0xc86a8e0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_genJet_tau_1_3_topSel': <cppyy.gbl.TH1F object at 0xc86ad40>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_missgenJet_tau_1_3_topSel': <cppyy.gbl.TH1F object at 0xc872250>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_accepgenJet_tau_1_3_topSel': <cppyy.gbl.TH1F object at 0xc86e180>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_truerecoJet_tau_1_3_nom_topSel': <cppyy.gbl.TH1F object at 0xc86ea40>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_truerecoJet_tau_1_3_nom_topSel_genBin': <cppyy.gbl.TH1F object at 0xeb5dfb0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_fakerecoJet_tau_1_3_nom_topSel': <cppyy.gbl.TH1F object at 0xeb5e800>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_fakerecoJet_tau_1_3_

{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau_2_4_nom_topSel': <cppyy.gbl.TH1F object at 0xbf84850>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau_2_4_nom_topSel_genBin': <cppyy.gbl.TH1F object at 0xd7704d0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_genJet_tau_2_4_topSel': <cppyy.gbl.TH1F object at 0xd798590>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_missgenJet_tau_2_4_topSel': <cppyy.gbl.TH1F object at 0xf28d2e0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_accepgenJet_tau_2_4_topSel': <cppyy.gbl.TH1F object at 0xd770b90>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_truerecoJet_tau_2_4_nom_topSel': <cppyy.gbl.TH1F object at 0xd791fc0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_truerecoJet_tau_2_4_nom_topSel_genBin': <cppyy.gbl.TH1F object at 0xf28c740>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_fakerecoJet_tau_2_4_nom_topSel': <cppyy.gbl.TH1F object at 0xd791640>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_fakerecoJet_tau_2_4_

Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file ../test/Results/topSel/UL17/BinningTests/_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_topSel_Purity_v1.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results/topSel/UL17/BinningTests/_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_topSel_Purity_v1.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: _tau_0p5_1can2D
Info in <TCanvas::Print>: pdf file ../test/Results/topSel/UL17/BinningTests/_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_topSel_responseMatrixv1.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results/topSel/UL17/BinningTests/_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_topSel_responseMatrixv1.png has been created
Warning in <TH2F::Sumw2>: Sum of squares of weights structure alrea

In [40]:
signalLabelBegin = 'TTToSemiLeptonic'
signalLabel = 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8'
inputFolder = '../test/Samples/'

sigFiles = {}

for iy in ( ['2018']):
    for isam in dictSamples:
        if not checkDict( isam, dictSamples )[iy]['skimmerHisto'].endswith('root'): continue
        if isam.startswith(signalLabelBegin) and ('erdON' not in isam) and ('hdamp' not in isam):
            sigFiles[isam] = [
                            ROOT.TFile.Open(inputFolder+checkDict( isam, dictSamples )[iy]['skimmerHisto'] ),
                            checkDict( isam, dictSamples )
                        ]
print (sigFiles)
variables=nSubVariables_topSel
runPurity(name='recoJet', outputFolder='../test/Results/topSel/UL18/BinningTests/', inputFiles=sigFiles, variables=nSubVariables_topSel, sel='_topSel', selection='_topSel', year=iy, version='v1', ext='pdf', lumi=1., process='MC')

{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8': [<cppyy.gbl.TFile object at 0xf2ef350>, {'2017': {'miniAOD': ['/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL17MiniAODv2-106X_mc2017_realistic_v9-v1/MINIAODSIM'], 'nanoAOD': ['/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/kadatta-RunIISummer19UL17PFNanoAOD-106X_mc2017_realistic_v9-v1-7a1edb72314467730e458def0bc98536/USER'], 'skimmer': ['/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/kadatta-RunIISummer19UL17PFNanoAOD-106X_mc2017_realistic_v9-v1-7a1edb72314467730e458def0bc98536/USER'], 'skimmerHisto': 'TTToSemileptonic_powheg_pythia8_2017UL.root', 'nevents': 1.0, 'nGenWeights': 32392787477.907993}, '2018': {'miniAOD': ['/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/RunIISummer19UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1/MINIAODSIM'], 'nanoAOD': ['/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/kadatta-RunIISummer19UL18PFNanoAOD-106X_upgrade2018_realistic_v16_L1v1-v1-754f8ab81f6f0298c5fa7c45094d30e4/USER'], 'ski

{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau_0p5_2_nom_topSel': <cppyy.gbl.TH1F object at 0xbd33870>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau_0p5_2_nom_topSel_genBin': <cppyy.gbl.TH1F object at 0xbd33d80>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_genJet_tau_0p5_2_topSel': <cppyy.gbl.TH1F object at 0xeb62430>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_missgenJet_tau_0p5_2_topSel': <cppyy.gbl.TH1F object at 0xeb62cf0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_accepgenJet_tau_0p5_2_topSel': <cppyy.gbl.TH1F object at 0xbc9b020>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_truerecoJet_tau_0p5_2_nom_topSel': <cppyy.gbl.TH1F object at 0xeb8d630>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_truerecoJet_tau_0p5_2_nom_topSel_genBin': <cppyy.gbl.TH1F object at 0xeb8da20>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_fakerecoJet_tau_0p5_2_nom_topSel': <cppyy.gbl.TH1F object at 0xbc977a0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_fake

{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau_1_3_nom_topSel': <cppyy.gbl.TH1F object at 0xbc532a0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau_1_3_nom_topSel_genBin': <cppyy.gbl.TH1F object at 0xbc53690>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_genJet_tau_1_3_topSel': <cppyy.gbl.TH1F object at 0xeb60100>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_missgenJet_tau_1_3_topSel': <cppyy.gbl.TH1F object at 0xbc52540>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_accepgenJet_tau_1_3_topSel': <cppyy.gbl.TH1F object at 0xeb5af30>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_truerecoJet_tau_1_3_nom_topSel': <cppyy.gbl.TH1F object at 0xeb5a980>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_truerecoJet_tau_1_3_nom_topSel_genBin': <cppyy.gbl.TH1F object at 0xbc856f0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_fakerecoJet_tau_1_3_nom_topSel': <cppyy.gbl.TH1F object at 0xeb58980>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_fakerecoJet_tau_1_3_

{'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau_2_4_nom_topSel': <cppyy.gbl.TH1F object at 0xb8dffe0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_recoJet_tau_2_4_nom_topSel_genBin': <cppyy.gbl.TH1F object at 0xb8dfbf0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_genJet_tau_2_4_topSel': <cppyy.gbl.TH1F object at 0xeb7dd80>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_missgenJet_tau_2_4_topSel': <cppyy.gbl.TH1F object at 0xc86ecc0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_accepgenJet_tau_2_4_topSel': <cppyy.gbl.TH1F object at 0xb8dccc0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_truerecoJet_tau_2_4_nom_topSel': <cppyy.gbl.TH1F object at 0xb8ddcf0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_truerecoJet_tau_2_4_nom_topSel_genBin': <cppyy.gbl.TH1F object at 0xb8dd150>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_fakerecoJet_tau_2_4_nom_topSel': <cppyy.gbl.TH1F object at 0xeb870a0>, 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_fakerecoJet_tau_2_4_

Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file ../test/Results/topSel/UL18/BinningTests/_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_topSel_Purity_v1.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results/topSel/UL18/BinningTests/_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_topSel_Purity_v1.png has been created
Info in <TCanvas::Print>: pdf file ../test/Results/topSel/UL18/BinningTests/_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_topSel_responseMatrixv1.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results/topSel/UL18/BinningTests/_tau_0p5_1_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8_topSel_responseMatrixv1.png has been created
Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canva

In [41]:
signalLabelBegin = 'TTJets'
signalLabel = 'TTJets_TuneCP5_13TeV-powheg-pythia8'
inputFolder = '../test/Samples/'

sigFiles = {}

for iy in ( ['2017']):
    for isam in dictSamples:
        if not checkDict( isam, dictSamples )[iy]['skimmerHisto'].endswith('root'): continue
        if isam.startswith(signalLabelBegin) and ('erdON' not in isam) and ('hdamp' not in isam):
            sigFiles[isam] = [
                            ROOT.TFile.Open(inputFolder+checkDict( isam, dictSamples )[iy]['skimmerHisto'] ),
                            checkDict( isam, dictSamples )
                        ]
print (sigFiles)
variables=nSubVariables_WSel
runPurity(name='recoJet', outputFolder='../test/Results/WSel/UL17/BinningTests/', inputFiles=sigFiles, variables=nSubVariables_WSel, sel='_WSel', selection='_WSel', year=iy, version='v1', ext='pdf', lumi=1., process='MC')

{'TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8': [<cppyy.gbl.TFile object at 0xbd2d7f0>, {'2017': {'miniAOD': ['/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer19UL17MiniAODv2-106X_mc2017_realistic_v9-v1/MINIAODSIM'], 'nanoAOD': ['/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/kadatta-RunIISummer19UL17PFNanoAOD-106X_mc2017_realistic_v9-v1-7a1edb72314467730e458def0bc98536/USER'], 'skimmer': ['/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/kadatta-RunIISummer19UL17PFNanoAOD-106X_mc2017_realistic_v9-v1-7a1edb72314467730e458def0bc98536/USER'], 'skimmerHisto': 'TTJets_amcatnloFXFX-pythia8_2017UL.root', 'nevents': 1.0, 'nGenWeights': 286629789811.7677}, '2018': {'miniAOD': ['/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/RunIISummer19UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1/MINIAODSIM'], 'nanoAOD': ['/TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8/kadatta-RunIISummer19UL18PFNanoAOD-106X_upgrade2018_realistic_v16_L1v1-v1-754f8ab81f6f0298c5fa7c45094d30e4/USER'], 'skimmer': ['/TTJets_TuneCP5_13Te

KeyError: 'TTJets_TuneCP5_13TeV-powheg-pythia8_accepgenJet_tau_0p5_1_WSel'

Warning in <TH2F::Sumw2>: Sum of squares of weights structure already created


In [ ]:
signalLabelBegin = 'TTJets'
signalLabel = 'TTJets_TuneCP5_13TeV-powheg-pythia8'
inputFolder = '../test/Samples/'

sigFiles = {}

for iy in ( ['2018']):
    for isam in dictSamples:
        if not checkDict( isam, dictSamples )[iy]['skimmerHisto'].endswith('root'): continue
        if isam.startswith(signalLabelBegin) and ('erdON' not in isam) and ('hdamp' not in isam):
            sigFiles[isam] = [
                            ROOT.TFile.Open(inputFolder+checkDict( isam, dictSamples )[iy]['skimmerHisto'] ),
                            checkDict( isam, dictSamples )
                        ]
print (sigFiles)
variables=nSubVariables_WSel
runPurity(name='recoJet', outputFolder='../test/Results/WSel/UL18/BinningTests/', inputFiles=sigFiles, variables=nSubVariables_WSel, sel='_WSel', selection='_WSel', year=iy, version='v1', ext='pdf', lumi=1., process='MC')

In [ ]:
signalLabelBegin = 'TTJets'
signalLabel = 'TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8'
inputFolder = '../test/Samples/'

sigFiles = {}

for iy in ( ['2017']):
    for isam in dictSamples:
        if not checkDict( isam, dictSamples )[iy]['skimmerHisto'].endswith('root'): continue
        if isam.startswith(signalLabelBegin) and ('erdON' not in isam) and ('hdamp' not in isam):
            sigFiles[isam] = [
                            ROOT.TFile.Open(inputFolder+checkDict( isam, dictSamples )[iy]['skimmerHisto'] ),
                            checkDict( isam, dictSamples )
                        ]
print (sigFiles)
variables=nSubVariables_topSel
runPurity(name='recoJet', outputFolder='../test/Results/topSel/UL17/BinningTests/', inputFiles=sigFiles, variables=nSubVariables_WSel, sel='_topSel', selection='_topSel', year=iy, version='v1', ext='pdf', lumi=1., process='MC')

In [ ]:
signalLabelBegin = 'TTJets'
signalLabel = 'TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8'
inputFolder = '../test/Samples/'

sigFiles = {}

for iy in ( ['2018']):
    for isam in dictSamples:
        if not checkDict( isam, dictSamples )[iy]['skimmerHisto'].endswith('root'): continue
        if isam.startswith(signalLabelBegin) and ('erdON' not in isam) and ('hdamp' not in isam):
            sigFiles[isam] = [
                            ROOT.TFile.Open(inputFolder+checkDict( isam, dictSamples )[iy]['skimmerHisto'] ),
                            checkDict( isam, dictSamples )
                        ]
print (sigFiles)
variables=nSubVariables_topSel
runPurity(name='recoJet', outputFolder='../test/Results/topSel/UL18/BinningTests/', inputFiles=sigFiles, variables=nSubVariables_topSel, sel='_topSel', selection='_topSel', year=iy, version='v1', ext='pdf', lumi=1., process='MC')